In [3]:
import pandas as pd

## - Check the dataset that is created now

In [2]:
# read the dataset that is created every day
df_products = pd.read_pickle('/home/ubuntu/Spitishop/Spitishop_RE/search_products_api.pkl')

In [3]:
df_products.head(2)

,Code,Price,Product_name,Product_name_eng,name,freq,categories,categories_text,categories_text_eng,attributes,attributes_text,attributes_text_eng,brand_name
0,29415,85.0,Πάπλωμα Κούνιας Πουπουλένιο Guy Laroche Bebe 100%,paploma kounias poupoulenio guy laroche bebe 100%,Βρεφικά,8.0,"[Βρεφικά, Παπλώματα Βρεφικά]",Βρεφικά Παπλώματα Βρεφικά,brefika paplwmata brefika,NaN,Guy Laroche,guy laroche,Guy Laroche
2,29609,10.0,Διακοσμητική Μαξιλαροθήκη (40x40) Guy Laroche ...,diakosmhtikh maxilarothhkh (40x40) guy laroche...,Σαλόνι,4.0,"[Σαλόνι, Διακοσμητικά Μαξιλάρια]",Σαλόνι Διακοσμητικά Μαξιλάρια,saloni diakosmhtika maxilaria,NaN,Guy Laroche,guy laroche,Guy Laroche


## - The process to create the dataset

In [4]:
# Import libraries
import MySQLdb
import pandas as pd
import datetime
import os

#!pip install elasticsearch --user
from elasticsearch import Elasticsearch
#!pip install python-greeklish --user
from greeklish.converter import Converter

In [5]:
# define some global greek text rules

# create a dict for some greek letters matching
greek_letters_dict = {'ω':'ο'}
# create an object for greeklish translation
conv = Converter(max_expansions=1)

In [24]:
print("Running products_search.py ", str(datetime.datetime.now()))

# read the password
p = pd.read_csv('/home/ubuntu/Spitishop/Spitishop_RE/p.txt', sep=" ", header=None).iloc[0,0]

print("Fetching data")
# connect to database
conn = MySQLdb.connect("195.201.212.160","infilab",p,"spitishop_redesign_2019", charset='utf8')

# Read the table with the category names
df_ps_category_lang = pd.read_sql("select * from ps_category_lang where id_lang=2 and id_shop=1", conn)

# read table with the product links
df_ps_product_lang = pd.read_sql("select id_product, name from ps_product_lang", conn)

# read the table with the categories of each product
df_ps_category_product = pd.read_sql("select * from ps_category_product", conn)

# df_ps_attribute = pd.read_sql("select * from ps_attribute", conn)
# df_ps_attribute_group = pd.read_sql("select * from ps_attribute_group", conn)
# df_ps_attribute_group_lang = pd.read_sql("select * from ps_attribute_group_lang", conn)
df_ps_attribute_lang= pd.read_sql("select * from ps_attribute_lang where id_lang=2", conn) #!


df_ps_product_attribute= pd.read_sql("select * from ps_product_attribute", conn) #!
df_ps_product_attribute_combination= pd.read_sql("select * from ps_product_attribute_combination", conn) #!
#df_ps_product_attribute_image= pd.read_sql("select * from ps_product_attribute_image", conn)
#df_ps_product_attribute_shop= pd.read_sql("select * from ps_product_attribute_shop", conn)

# read the manufacturer of each product
df_ps_product = pd.read_sql("select id_product, id_manufacturer, reference from ps_product", conn)

# read the manufacturer names
df_ps_manufacturers = pd.read_sql("select * from ps_manufacturer", conn)

#df_tables = pd.read_sql("show tables", conn)

# Close connection
conn.close()

print("Data fetched")


# read the dataset with the products
products = pd.read_pickle('/home/ubuntu/Spitishop/Spitishop_RE/products_api.pkl')

Running products_search.py  2020-09-23 15:26:34.729079
Fetching data
Data fetched


In [40]:
products = pd.read_pickle('/home/ubuntu/Spitishop/Spitishop_RE/products_api.pkl')

In [41]:
products.head()

,id_product,id_category_default,seasonality,price,id_category_1,id_category_2,id_category_3,id_category_4,categories,freq
0,29415,200030.0,"[3, 4]",85.000000,NaN,1004.0,200030.0,NaN,"[1004, 200030.0]",8.0
1,29609,5000211.0,[],10.000000,NaN,1001.0,5000211.0,NaN,"[1001, 5000211.0]",4.0
2,29611,5000211.0,[],10.000000,NaN,1001.0,5000211.0,NaN,"[1001, 5000211.0]",4.0
3,29675,5000295.0,[],21.999999,NaN,1002.0,20007.0,5000295.0,"[1002, 20007.0, 5000295.0]",8.0
4,29865,200014.0,[],59.000000,NaN,1005.0,200014.0,NaN,"[1005, 200014.0]",4.0


In [45]:
products.id_category_2.nunique()

20

In [25]:
# create a df with brand name of each product

df_brands = df_ps_product.merge(df_ps_manufacturers[['id_manufacturer', 'name']], left_on='id_manufacturer', right_on='id_manufacturer', how='left')
# drop duplicates
df_brands = df_brands.drop_duplicates(subset=['id_product', 'name'], keep='last')

# keep specific columns
df_brands=df_brands[['id_product', 'name', 'reference']]
df_brands.columns = ['id_product', 'brand_name', 'reference']

In [27]:
# create a df with the names of all the attributes of each product (as a list)

# add attribute code of each product
df_attribute = df_ps_product_attribute[['id_product', 'id_product_attribute']].merge(df_ps_product_attribute_combination, left_on='id_product_attribute', right_on='id_product_attribute', how='left')
# add the name of each attribute
df_attribute = df_attribute.merge(df_ps_attribute_lang, left_on='id_attribute', right_on='id_attribute', how='left')
# keep only unique attributes for each product
df_attribute = df_attribute.drop_duplicates(subset=['id_product', 'name'], keep='first')
# group by and create a list of the attribute names of each product
df_product_attributes = df_attribute.groupby('id_product')['name'].apply(list).reset_index(name='attributes')
# create a list with all the attributes as a concatenated text
df_product_attributes['attributes_text'] = df_product_attributes.apply(lambda x: ' '.join(x['attributes']), axis=1)

In [28]:
# create a df with the names of all the categories of each product (as a list)

# add the name of each category
df_ps_category_product = df_ps_category_product.merge(df_ps_category_lang[['id_category','name']], left_on='id_category', right_on='id_category', how='left')
# keep only unique attributes for each product
df_ps_category_product = df_ps_category_product.drop_duplicates(subset=['id_product', 'name'], keep='first')
# group by and create a list of the category names of each product
df_product_categs = df_ps_category_product.groupby('id_product')['name'].apply(list).reset_index(name='categories')
# create a list with all the categories as a concatenated text
df_product_categs['categories_text'] = df_product_categs.apply(lambda x: ' '.join(x['categories']), axis=1)

In [29]:
# create a column with the basic category id
products['category'] = products.apply(lambda x: x['categories'][0], axis=1)
# keep specific columns
products_search1 = products[['id_product', 'price', 'freq', 'category']]

# add the category names
products_search1 = products_search1.merge(df_ps_category_lang[['id_category','name']], left_on='category', right_on='id_category', how='left')

# add the product titles
products_search1 = products_search1.merge(df_ps_product_lang, left_on='id_product', right_on='id_product', how='left')

# create a column with a list of all the category names
products_search1 = products_search1.merge(df_product_categs, left_on='id_product', right_on='id_product', how='left')

# create a column with a list of all the attributes names
products_search1 = products_search1.merge(df_product_attributes, left_on='id_product', right_on='id_product', how='left')

# create a column with the brand name of each product
products_search1 = products_search1.merge(df_brands, left_on='id_product', right_on='id_product', how='left')

# rename columns
products_search1.columns = ['Code', 'Price', 'freq', 'category_id', 'id_category', 'name','Product_name', 'categories', 'categories_text', 'attributes', 'attributes_text', 'brand_name', 'reference']

# convert nans to ''
products_search1['attributes_text'] = products_search1['attributes_text'].fillna(' ')
# add the brand of each product to the 'attributes_text'
products_search1['attributes_text'] = products_search1['brand_name'] +' '+ products_search1['attributes_text']

# create a copy of the dataframe
df_products = products_search1.copy()

# convert Product_name to lowercase, greeklish etc
# create a column with the title having changed some letters based on greek_letters_dict
df_products['Product_name2'] = df_products.apply(lambda x: x["Product_name"].lower().translate(str.maketrans(greek_letters_dict)), axis=1)
# create a column with the greeklish title
df_products['Product_name_eng'] = df_products.apply(lambda x: conv.convert(x["Product_name2"])[0], axis=1)

# convert categories_text to lowercase, greeklish etc 
# create a column with the categories_text having changed some letters based on greek_letters_dict
df_products['categories_text2'] = df_products.apply(lambda x: x["categories_text"].lower().translate(str.maketrans(greek_letters_dict)), axis=1)
# create a column with the greeklish title
df_products['categories_text_eng'] = df_products.apply(lambda x: conv.convert(x["categories_text2"])[0], axis=1)

# convert attributes_text to lowercase, greeklish etc 
# create a column with the attributes_text having changed some letters based on greek_letters_dict
df_products['attributes_text2'] = df_products.apply(lambda x: x["attributes_text"].lower().translate(str.maketrans(greek_letters_dict)), axis=1)
# create a column with the greeklish title
df_products['attributes_text_eng'] = df_products.apply(lambda x: conv.convert(x["attributes_text2"])[0], axis=1)

# keep only specific columns
df_products = df_products[['Code', 'Price', 'Product_name', 'Product_name_eng', 'name', 'freq', 'categories', 'categories_text', 'categories_text_eng', 'attributes', 'attributes_text', 'attributes_text_eng', 'brand_name', 'reference']]

# drop rows with nans
#df_products = df_products.dropna()

# drop duplicates
df_products = df_products.drop_duplicates(subset=['Code'], keep='first')

In [32]:
df_products.head(2)

,Code,Price,Product_name,Product_name_eng,name,freq,categories,categories_text,categories_text_eng,attributes,attributes_text,attributes_text_eng,brand_name,reference
0,29415,85.0,Πάπλωμα Κούνιας Πουπουλένιο Guy Laroche Bebe 100%,paploma kounias poupoulenio guy laroche bebe 100%,Βρεφικά,8.0,"[Βρεφικά, Παπλώματα Βρεφικά]",Βρεφικά Παπλώματα Βρεφικά,brefika paplwmata brefika,NaN,Guy Laroche,guy laroche,Guy Laroche,000000700022
2,29609,10.0,Διακοσμητική Μαξιλαροθήκη (40x40) Guy Laroche ...,diakosmhtikh maxilarothhkh (40x40) guy laroche...,Σαλόνι,4.0,"[Σαλόνι, Διακοσμητικά Μαξιλάρια]",Σαλόνι Διακοσμητικά Μαξιλάρια,saloni diakosmhtika maxilaria,NaN,Guy Laroche,guy laroche,Guy Laroche,1116030113008


In [1]:
df_products.to_pickle("data/df_products_new.pkl")

NameError: name 'df_products' is not defined

In [4]:
df_products = pd.read_pickle("data/df_products_new.pkl")

In [5]:
df_products.name.unique()

array(['Βρεφικά', 'Σαλόνι', 'Κρεβατοκάμαρα', 'Μπάνιο', 'Παιδικά',
       'Κουρτίνες', 'Κουζίνα - Τραπεζαρία', 'Μόδα Γάμου', 'Χαλιά',
       'Προώθηση', 'Θαλάσσης', 'Διακόσμηση', 'Χριστουγεννιάτικα',
       'Παιχνίδια', 'Επαγγελματικός Εξοπλισμός', 'Φωτιστικά',
       'Δώρα για νεογέννητο', 'Κήπος', 'Πασχαλινά',
       'Υγιεινή - Μάσκες Προστασίας'], dtype=object)

In [6]:
df_products.name.nunique()

20

## -- Create a new elastic search index with the new dataset

In [18]:
# # create a list of dicts with the product details
# products = []
# tmp = df_products.apply(lambda x: products.append({'title_eng':x['Product_name_eng'],
#                                                          'title_init':x['Product_name'],
#                                                          #'url':x['Url'],
#                                                          'Price':x['Price'],
#                                                          #'Brand':x['Brand'],
#                                                          'Category':x['name'],
#                                                          #'img_url':x['img_url'],
#                                                          'Code':x['Code'],
#                                                          'Popularity':x['freq']
#                                                   }), axis=1)

# # connect to elasticsearch / create an Elasticsearch object
# def connect_elasticsearch():
#     _es = None
#     #_es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
#     _es = Elasticsearch()
#     if _es.ping():
#         print('- Connected')
#     else:
#         print('- Problem with the connection')
#     return _es

# es = connect_elasticsearch()

# # delete the old index
# es.indices.delete(index='spitishop_1', ignore=[400, 404])

# def create_index(es_object, index_name='spitishop_1'):
    
#     # index settings
# #     settings = {
# #           "settings": {
# #             "analysis": {
# #               "analyzer": {
# #                 "my_analyzer": { 
# #                   "type": "standard", # https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-standard-analyzer.html
# #                   "stopwords": ["sentoni", "the", 'mpournouzi'],
# #                   #"stopwords_path": "stopwords/my_stopwords.txt"
# #                 }
# #               }
# #             }
# #           },
# #         #"mappings": {}
# #     }

#     # https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-synonym-tokenfilter.html
#     settings = {
#                 "settings": {
#                     "index" : {
#                         "analysis" : {
#                             "analyzer" : {
#                                 "my_analyzer" : {
#                                     "tokenizer" : "standard",
#                                     "filter" : ["my_stop", "synonym"]
#                                 }
#                             },
#                             "filter" : {
#                                 "my_stop": {
#                                     "type" : "stop",
#                                     "stopwords": ["to", "kai"]
#                                 },
#                                 "synonym" : {
#                                     "type" : "synonym",
#                                     "lenient": True,
#                                     "synonyms" : ["ypnodomatio, bedroom => krebatokamara"]
#                                 }
#                             }
#                         }
#                     }
#                 }
#             }
    
#     try:
#         #if not es_object.indices.exists(index_name):
#         # Ignore 400 means to ignore "Index Already Exist" error.
#         es_object.indices.create(index=index_name, ignore=400, body=settings)
#         print('Created Index')
#     except Exception as ex:
#         print('Error: ', str(ex))

# create_index(es, 'spitishop_1')

# # add products to the index
# i = 1
# for product in products[0:]:
#     es.index(index='spitishop_1',
#              doc_type='product',
#              id=i,
#              body=product,
#             )
#     i += 1

# df_products.to_pickle('/home/ubuntu/Spitishop/Spitishop_RE/search_products_api.pkl')

# print("Copy the pickle file to the API folder")
# # move the pickle files to the API file
# os.system("sudo cp /home/ubuntu/Spitishop/Spitishop_RE/search_products_api.pkl /home/spitishop/spitishop_rec_app")

# # write a log message
# with open("/home/ubuntu/Spitishop/Spitishop_RE/logs/search_products.txt", "a") as f:
#     f.write("Run properly at: %s \n" % str(datetime.datetime.now()))

In [82]:
#!/opt/anaconda3/bin/ipython /home/ubuntu/Spitishop/Spitishop_RE/products_search.py

]0;IPython: search_engine/TestingRunning products_search.py  2020-09-21 11:06:15.396481
Fetching data
Data fetched
- Connected
Created Index
Copy the pickle file to the API folder
